In [1]:
from pandas import DataFrame, read_csv
from string_utils import morph_parse, make_tokens
from gensim.models import Word2Vec
from scipy import spatial
import numpy as np
from sklearn.metrics import f1_score

In [2]:
df = read_csv('annotated.csv', encoding='cp1251')
df.comment = df.comment.apply(morph_parse)
df.reference = df.reference.apply(morph_parse)
df['labels'] = df['labels'].astype(int)

In [3]:
df['labels'].value_counts()

0    781
1    739
Name: labels, dtype: int64

In [4]:
word2vec = Word2Vec.load('all_lem_100')
word2vec_vocab = word2vec.wv.vocab

In [5]:
def similarity(message, reference, vocab, num_features=100):
    featureVec1 = np.zeros((num_features), dtype='float32')
    for word in make_tokens(message.lower(), vocab):
        if word in vocab: 
            featureVec1 = np.add(featureVec1, word2vec[word])
    featureVec2 = np.zeros((num_features), dtype='float32')
    for word in make_tokens(reference.lower(), vocab):
        if word in vocab: 
            featureVec2 = np.add(featureVec2, word2vec[word])
    return 1 - spatial.distance.cosine(featureVec1, featureVec2)

In [6]:
results = np.zeros(len(df), dtype='float32')

for i, m in df.iterrows():
    results[i] = similarity(m['comment'], m['reference'], word2vec_vocab)

/home/defeater/.local/lib/python3.5/site-packages/scipy/spatial/distance.py:505: RuntimeWarning: invalid value encountered in true_divide
  dist = 1.0 - np.dot(u, v) / (norm(u) * norm(v))


In [7]:
results = [1 if i > 0.3 else 0 for i in results]

In [8]:
f1_score(df['labels'], results)

0.62940190050307432